## Improvement1: find best depth_limit

We use train_full to train and validation to choose the best depth_limit

In [1]:
import numpy as np
from classification import *
from util import Util
from improvement import *

In [2]:
train_x, train_y = Util.read_dataset("./data/train_full.txt")
val_x, val_y = Util.read_dataset("./data/validation.txt")
test_x, test_y = Util.read_dataset("./data/test.txt")

# best_depth = 0.
# best_val_accuracy = 0.

# print(f"Best depth is chose according to val accuracy")
# print(f"Test accuracy is just illustrated")
# print(f"It is not used in model training")

# print()
# for depth in range(30):
#     print(f"depth: {depth}", end = " ")
#     DecisionTreeClassifier.depth_limit = depth
#     classifier = DecisionTreeClassifier()
#     classifier.fit(train_x, train_y)
#     prediction = classifier.predict(val_x)
#     val_accuracy = Evaluation.accuracy(val_y, prediction)
#     print(f"val accucary: {val_accuracy:.3f}", end = " ")
#     prediction = classifier.predict(test_x)
#     test_accuracy = Evaluation.accuracy(test_y, prediction)
#     print(f"test accucary: {test_accuracy:.3f}")
#     if val_accuracy > best_val_accuracy:
#         best_val_accuracy = val_accuracy
#         best_depth = depth
        
# print(f"best depth = {best_depth}")
# print(f"best val accuracy = {best_val_accuracy}")
# DecisionTreeClassifier.depth_limit = best_depth
# classifier = DecisionTreeClassifier()
# classifier.fit(train_x, train_y)
# prediction = classifier.predict(test_x)
# accuracy = Evaluation.accuracy(test_y, prediction)
# print(f"test accuracy for a best depth deicision tree = {accuracy}")
# print(f"From now on, the depth of the classifier will be set to {best_depth}")

In [3]:
classifier = DecisionTreeClassifier()
classifier.fit(train_x, train_y)
prediction = classifier.predict(test_x)
accuracy = Evaluation.accuracy(test_y, prediction)
print(f"test accuracy for a best depth deicision tree = {accuracy}")

test accuracy for a best depth deicision tree = 0.865


## Improvement2: Prune

In [4]:
train_val_x = np.concatenate((train_x, val_x), axis=0)
train_val_y = np.concatenate((train_y, val_y), axis=0)

print(f"common best depth deicision tree: ")
common_node_list, best_common_node = CrossValidation.cross_validation(train_val_x, train_val_y, 10)
common_classifier = DecisionTreeClassifier(best_common_node)
predictions = common_classifier.predict(test_x)
accuracy = Evaluation.accuracy(test_y, prediction)
print(f"Test accuracy for a common best depth deicision tree = {accuracy}\n")

print(f"pruned best depth deicision tree: ")
pruned_node_list, best_pruned_node = Prune.prune(train_val_x, train_val_y, 10)
pruned_classifier = DecisionTreeClassifier(best_pruned_node)
predictions = pruned_classifier.predict(test_x)
accuracy = Evaluation.accuracy(test_y, prediction)
print(f"Test accuracy for a pruned best depth deicision tree = {accuracy}")

common best depth deicision tree: 
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A' 'C' 'E' 'G' 'O' 'Q']
['A

## Vote using the pruned, best-depth decision tree


In [5]:
accuracies = np.zeros((10, ))
x_test, y_test = Util.read_dataset("./data/test.txt")
prediction_all = np.repeat(np.zeros_like(y_test).reshape(-1,y_test.size), 10, axis=0)
avg_acc = 0

for i in range(10):
    prediction = DecisionTreeClassifier(pruned_node_list[i]).predict(x_test)
    prediction_all[i,:] = prediction
    avg_acc += Evaluation.accuracy(prediction, y_test)/10
prediction_votes = np.zeros_like(y_test)

for i in range(y_test.size):
    votes = prediction_all[:,i]
    unique, counts = np.unique(votes, return_counts=True)
    prediction_votes[i] = unique[counts.argmax()]
    # optional: print all difference
    if (unique.size > 1):
        print(votes, end = " ")
        print("vote for: ", prediction_votes[i])
        
vote_accuracy = Evaluation.accuracy(prediction_votes, y_test)
print(f"vote accuracy of pruned, best-depth decision trees is {vote_accuracy:.4f}")

['Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'O' 'Q' 'O'] vote for:  Q
['E' 'E' 'E' 'C' 'E' 'E' 'E' 'C' 'E' 'E'] vote for:  E
['Q' 'Q' 'Q' 'Q' 'O' 'O' 'Q' 'O' 'O' 'Q'] vote for:  Q
['G' 'G' 'G' 'G' 'G' 'G' 'E' 'G' 'G' 'G'] vote for:  G
['G' 'G' 'G' 'Q' 'G' 'E' 'G' 'E' 'E' 'E'] vote for:  G
['O' 'O' 'O' 'Q' 'Q' 'O' 'O' 'O' 'Q' 'O'] vote for:  O
['O' 'O' 'O' 'O' 'O' 'O' 'Q' 'O' 'O' 'O'] vote for:  O
['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'Q'] vote for:  O
['E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'G'] vote for:  E
['Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'O' 'O'] vote for:  Q
['Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'O' 'Q' 'Q' 'Q'] vote for:  Q
['C' 'A' 'A' 'A' 'C' 'A' 'A' 'C' 'C' 'A'] vote for:  A
['O' 'Q' 'O' 'Q' 'Q' 'O' 'Q' 'Q' 'Q' 'Q'] vote for:  Q
['A' 'A' 'A' 'A' 'O' 'A' 'A' 'A' 'A' 'A'] vote for:  A
['Q' 'O' 'Q' 'Q' 'O' 'G' 'Q' 'Q' 'O' 'O'] vote for:  Q
['G' 'G' 'G' 'O' 'G' 'E' 'G' 'G' 'E' 'E'] vote for:  G
['O' 'G' 'O' 'O' 'O' 'O' 'O' 'C' 'O' 'G'] vote for:  O
['A' 'G' 'G' 'Q' 'G' 'C' 'A' 'A' 'G' 'O'] vote for:  G
['E' 'E' '